In [ ]:
import json
import requests


In [ ]:
with open("data/K2-00001-CL33762-CP33206-05-08-S2.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)


In [ ]:
def build_prompt(data, topic):
    session = data["sessionInfo"]

    # 프로필 및 페르소나 구성
    profile1 = f'{data["participantsInfo"]["speaker1"]["age"]} {data["participantsInfo"]["speaker1"]["gender"]}, {data["participantsInfo"]["speaker1"]["occupation"]}, 출신: {data["participantsInfo"]["speaker1"]["bPlace"]}'
    profile2 = f'{data["participantsInfo"]["speaker2"]["age"]} {data["participantsInfo"]["speaker2"]["gender"]}, {data["participantsInfo"]["speaker2"]["occupation"]}, 출신: {data["participantsInfo"]["speaker2"]["bPlace"]}'
    persona1 = ", ".join(data["personaInfo"]["clInfo"]["personaFeatures"])
    persona2 = ", ".join(data["personaInfo"]["cpInfo"]["personaFeatures"])

    # 대화 이력 구성
    turns = []
    for s in session:
        for d in s["dialog"]:
            speaker = d["speaker"]
            utterance = d["utterance"]
            turns.append(f"{speaker}: {utterance}")
    dialog_history = "\n".join(turns)

    # 최종 프롬프트 구성
    prompt = f"""Topic: {topic}

## Speaker 1
- Profile: {profile1}
- Persona: {persona1}
- Style: polite, emotionally expressive

## Speaker 2
- Profile: {profile2}
- Persona: {persona2}
- Style: concise, rational

## Rules:
- Do not restart or summarize. Continue from the last turn.
- Use Korean for all dialogue output.
- Reflect personalities in tone and word choice.
- Avoid politics, religion, or offensive topics.
- Encourage friendly, open-ended dialogue.
- If possible, end with a natural question or reply hook.

## Previous Dialogue:
{dialog_history}

Write the next 1–2 turns of conversation in Korean like this:
speaker1: ...
speaker2: ..."""

    return prompt


In [ ]:
topic = "편지"  # 또는 data["topicInfo"]["topicTitle"]
prompt_input = build_prompt(raw_data, topic)

final_input = (
    f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>"
    f"You are a Korean dialogue generation model. Continue a realistic and coherent long-form conversation between two speakers. Use their profile, personality traits (persona), and previous conversation history to write 1–2 additional turns in Korean. Match each speaker’s tone and perspective. Maintain natural flow and avoid repetition or restarting the dialogue."
    f"<|eot_id|><|start_header_id|>user<|end_header_id|>{prompt_input}<|eot_id|>"
    f"<|start_header_id|>assistant<|end_header_id|>"
)



In [ ]:
url = "http://114.70.193.164:8001/v1/completions"
headers = {"Content-Type": "application/json"}

payload = {
    "model": "ibnzterrell/Meta-Llama-3.3-70B-Instruct-AWQ-INT4",
    "prompt": final_input,
    "max_tokens": 512
}

response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    result = response.json()
    print(result["choices"][0]["text"])
else:
    print(f"❌ Error {response.status_code}: {response.text}")


In [ ]:
output = {
    "input_prompt": prompt_input,
    "llama_response": result["choices"][0]["text"]
}

with open("outputs/augmented/K2-00001-augmented.json", "w", encoding="utf-8") as f:
    json.dump(output, f, ensure_ascii=False, indent=2)
